Import Library

In [4]:
from twython import Twython
import sqlite3
from time import sleep
import numpy as np
import requests
import json
import pandas as pd
import io
import pickle
from tqdm import tqdm
import warnings
from util import JSONParser
warnings.simplefilter(action='ignore', category=FutureWarning)

Access API

In [2]:
import tweepy

# Authenticate to Twitter
auth = tweepy.OAuthHandler("CkhGa0z1CKMlK7GY56Zxh0250", 
    "Iq1o1IwRzuFw6xCandQbGUX1gzx6GezPD7ek1ptbcw46eni3LB")
auth.set_access_token("492948056-n4H0mlG3efeuJnDKkkoyeW9BWbl7kBrI95Dmlup1", 
    "jMSNiTz1mo764Xnz1x8cEAuu2CQRIhRbstAgRiD5WY7cP")

api = tweepy.API(auth)

try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

Authentication OK


In [18]:
jp=JSONParser()
jp.parse("data/intents.json")

Important Function

In [ ]:
def open_model(model_path):
    with open (model_path, 'rb') as f:
        model = pickle.load(f)
    return model
model=open_model("pipe.pkl")

In [ ]:
def response(data):
    tag=model.predict(data)
    response=[]
    for t in tag:
        response.append(jp.get_response(t))
    return response


In [6]:
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
    return conn

In [7]:
def get_massage(api):
    pesan=api.get_direct_messages()
    df= pd.DataFrame([[s.id, s.created_timestamp, s.message_create['message_data']['text'], s.message_create['target']['recipient_id'], s.message_create['sender_id']] for s in pesan], columns=('ID', 'created at', 'text', 'recipient', 'sender' ))
    return df[df.sender!='492948056']

In [15]:
URL = 'http://tf-serving-sentiment.herokuapp.com/v1/models/sentiment:predict'
def get_predict(series, URL_Serving):
    X=[]
    for con in series.tolist():
        X.append([con])

    input_data_json = json.dumps({
        "signature_name": "serving_default",
        "instances": X,
        })
        
    r=requests.post(URL_Serving, input_data_json)
    result=r.json()
    result= np.argmax(result['predictions'], axis=1)
    return np.where(result==0,'Negative', np.where(result==1, 'Neutral', 'Positive'))

In [46]:
def check_new_massage(api):
    old_massage=get_massage(api)
    while True:
        max=old_massage.ID.max()
        new_massage=get_massage(api)
        if new_massage.ID.max()>old_massage.ID.max():
            massage=new_massage[new_massage.ID>max]
            massage['response']=response(massage['text'])
            print(f"Ada {massage.shape[0]} pesan baru")
            old_massage=new_massage
            for mas in massage.ID:
                api.send_direct_message(massage[massage.ID==mas].sender[0], massage[massage.ID==mas].response[0])
        else:
            print("tidak ada pesan baru")
        sleep(65)

Connection

In [8]:
con = create_connection("twit copy.db")
cursor=con.cursor()

In [48]:
check_new_massage(api)

TooManyRequests: 429 Too Many Requests
88 - Rate limit exceeded